# NLP Assignment 2 (40% of grade): Vector Space Semantics for Similarity between EastEnders Characters

In this assignment, you will be creating a vector representation of a document containing lines spoken by a character in the EastEnders script data (i.e. from the file `training.csv`), then improving that representation such that each character vector is maximally distinguished from the other character documents. This distinction is measured by how well a simple information retrieval classification method can select documents from validation and test data as belonging to the correct class of document (i.e. deciding which character spoke the lines by measuring the similarity of those character document vectors to those built in training).

As the lines are not evenly distributed in terms of frequency, this coursework stipulates you can only use a maximum of the first **300 lines** of each character in the training data `training.csv` to create the training documents and a maximum of the first **50 lines** in the validation and test data (from `val.csv` and `test.csv`). This makes it more challenging, as the number of lines spoken by a character can't be used directly or otherwise as a feature.

A simple vector representation for each character document is done for you to start with in this code, as is the pipeline of similarity-based information retrieval evaluation. You need to improve the character vector representations by pre-processing, feature extraction and transformation techniques, as per Questions 1-6 below, which you need to complete as instructed.

**Refer to the material in units 8-9 for conceptual background.**

In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import string
import re
import numpy as np
from numpy.linalg import norm
import pandas as pd
from collections import Counter, OrderedDict

import seaborn as sns
import matplotlib.pyplot as plt

import math

import nltk

from sklearn.feature_extraction import DictVectorizer

%matplotlib inline
pd.options.display.max_colwidth=500

In [3]:
# Load in training data and display in pandas dataframe
train_path='training.csv'
train_data = pd.read_csv(train_path,  delimiter="\t", skip_blank_lines = True)
val_path ='val.csv'
val_data = pd.read_csv(val_path,  delimiter="\t", skip_blank_lines = True)
test_path ='test.csv'
test_data = pd.read_csv(test_path,  delimiter="\t", skip_blank_lines = True)

# Inspect
train_data

Episode  Scene                   Scene_info Character_name  \
0         1350      1  DESERTED CAR PARK EXT NIGHT        SHIRLEY   
1         1350      1  DESERTED CAR PARK EXT NIGHT          OTHER   
2         1350      2                R&R INT NIGHT           JACK   
3         1350      2                R&R INT NIGHT         RONNIE   
4         1350      2                R&R INT NIGHT           JACK   
...        ...    ...                          ...            ...   
14008     1399     55         SQUARE EXT DAY LIGHT          OTHER   
14009     1399     55         SQUARE EXT DAY LIGHT            MAX   
14010     1399     55         SQUARE EXT DAY LIGHT          OTHER   
14011     1399     55         SQUARE EXT DAY LIGHT            MAX   
14012     1399     55         SQUARE EXT DAY LIGHT          OTHER   

                                                                                                              Line  \
0                                               Look at ya, not a mark on ya. And you think you're an unlucky man.   
1                                                                                                         Shirl...   
2                                                         Oi. Where have you been? Huh? What were the texts about?   
3                                                          Nothing. Nothing. I'll be with you in two minutes yeah?   
4            Well I've got mates here I wanted to have a chat with them, instead I've been serving behind the bar.   
...                                                                                                            ...   
14008  Dad? Okay ... alright, just one drink alright. But that's all. It doesn't mean anything. It's just a drink.   
14009                                                   Thanks Bradley. Thanks mate... It means the world to me...   
14010                                                                                               You alright...   
14011                                                                                  Yeah, yeah, yeah. I'm fine.   
14012                                                                                           Well, let me know.   

       Gender  
0      FEMALE  
1        MALE  
2        MALE  
3      FEMALE  
4        MALE  
...       ...  
14008    MALE  
14009    MALE  
14010    MALE  
14011    MALE  
14012    MALE  

[14013 rows x 6 columns]

In [5]:
# # Create one document per character
# def create_character_document_from_dataframe(df, max_line_count):
#     """Returns a dict with the name of the character as key,
#     their lines joined together as a single string, with end of line _EOL_
#     markers between them

#     Improve this for Q3
    
#     ::max_line_count:: the maximum number of lines to be added per character
#     """
#     character_docs = {}
#     character_line_count = {}
#     for line, name, gender in zip(df.Line, df.Character_name, df.Gender):
#         #remove the empty lines
#         if (isinstance(line, float) and math.isnan(line)) or line == "":
#             continue
#         if not name in character_docs.keys():
#             character_docs[name] = ""
#             character_line_count[name] = 0
#         if character_line_count[name]==max_line_count:
#             continue
#         character_docs[name] += str(line)   + " _EOL_ "  # adding an end-of-line token
#         character_line_count[name]+=1
#     print("lines per character", character_line_count)
#     return character_docs




import math

def create_character_docs(df, max_line_count):
    
    character_docs = {}
    character_line_count = {}

    for (episode, scene), scene_data in df.groupby(['Episode', 'Scene']):
        scene_data = scene_data.sort_index()
        scene_characters = set(scene_data['Character_name'])

        for _, row in scene_data.iterrows():
            line = row['Line']
            name = row['Character_name']

            # Robust line cleaning
            if isinstance(line, float) and math.isnan(line):
                continue
            line = str(line).strip()  # Remove leading/trailing whitespace
            if not line:
                continue

            # Contextual filtering: Only include lines if the character interacts
            # with at least one other character in the scene
            if len(scene_characters) > 1:
                if name not in character_docs:
                    character_docs[name] = ""
                    character_line_count[name] = 0
                if character_line_count[name] < max_line_count:
                    character_docs[name] += line + " EOL "
                    character_line_count[name] += 1

    print("lines per character", character_line_count)
    return character_docs

# def create_character_docs(df, max_line_count):
#     """
#     Returns a dict with the name of the character as key,
#     their lines joined together as a single string, with end-of-line (EOL)
#     markers between them. Incorporates dialogue context and scene features.
#     ::max_line_count:: the maximum number of lines to be added per character
#     """
#     character_docs = {}
#     character_line_count = {}
#     # Group the dataframe by Episode and Scene for precise grouping
#     grouped_by_scene = df.groupby(['Episode', 'Scene'])
#     for line, name, scene, episode in zip(df.Line, df.Character_name, df.Scene, df.Episode):
#         # Skip empty or NaN lines
#         if (isinstance(line, float) and math.isnan(line)) or line == "":
#             continue
#         # Initialize character_docs entry if it doesn't exist
#         if name not in character_docs:
#             character_docs[name] = ""
#             character_line_count[name] = 0
#         # Add lines only from the same scene
#         if (episode, scene) in grouped_by_scene.groups:
#             scene_lines = grouped_by_scene.get_group((episode, scene)).Line.tolist()
#             # Add context lines (before and after) for the target line
#             context_lines = []
#             line_index = scene_lines.index(line)
#             # Add lines before the target character's line
#             context_lines.extend(scene_lines[max(0, line_index - 2):line_index])
#             # Add the target character's line
#             context_lines.append(line)
#             # Add lines after the target character's line
#             context_lines.extend(scene_lines[line_index + 1:min(line_index + 3, len(scene_lines))])
#             # Join context lines and add an end-of-line token
#             character_docs[name] += " ".join(map(str, context_lines)) + " EOL "
#             character_line_count[name] += 1
#         # Stop adding lines once the maximum count is reached for a character
#         if character_line_count[name] == max_line_count:
#             continue
#     print("Lines per character:", character_line_count)
#     return character_docs

In [ ]:
# print out the number of words each character has in the training set
# only use the first 300 lines of each character
train_character_docs = create_character_docs(train_data, max_line_count=300)
print('Num. Characters: ',len(train_character_docs.keys()),"\n")
total_words = 0
for name in train_character_docs.keys():
    print(name, 'Number of Words: ',len(train_character_docs[name].split()))
    total_words += len(train_character_docs[name].split())
print("total words", total_words)

In [7]:
# def pre_process(character_text):
#     """Pre-process all the concatenated lines of a character, 
#     using tokenization, spelling normalization and other techniques.
    
#     Initially just a tokenization on white space. Improve this for Q1.
    
#     ::character_text:: a string with all of one character's lines
#     """
#     tokens = character_text.split()   # just a simple tokenization, to be replaced
#     return tokens



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import string

def pre_process(character_text):
    """Pre-process all the concatenated lines of a character."""
    # 1. Lowercasing
    character_text = character_text.lower()

    # 2: Separate the punctuations from words
    character_text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", character_text)
    character_text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", character_text)

    # 3. Tokenization (simple whitespace split)
    tokens = character_text.split()

    # 4. Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Punctuation Removal
    tokens = [word for word in tokens if word not in string.punctuation]

    # # Ensure tokens are not empty
    # if not tokens:
    #     tokens = ['placeholder']

    # # 6. Adding Bigrams and Trigrams
    # bigrams = list(ngrams(tokens, 2))
    # trigrams = list(ngrams(tokens, 3))

    # # Flatten bigrams and trigrams into a list of strings
    # bigrams = [' '.join(bigram) for bigram in bigrams]
    # trigrams = [' '.join(trigram) for trigram in trigrams]

    # # Combine bigrams and trigrams with tokens
    # tokens += bigrams
    # tokens += trigrams

    return tokens

In [8]:
# create list of pairs of (character name, pre-processed character) 
training_corpus = [(name, pre_process(doc)) for name, doc in sorted(train_character_docs.items())]
train_labels = [name for name, doc in training_corpus]

In [21]:
# def to_feature_vector_dictionary(character_doc, extra_features):
#     """Converts a list of pre-processed tokens and extra features
#     to a Dictionary as a function of the tokens.
    
#     Initially just a simple count. Improve this for Q2.
    
#     ::character_doc:: a list of pre-processed tokens
#     ::extra_features:: any extra features for the character to be added to feature vector dict
#     """
#     counts = Counter(character_doc)  # for now a simple count
#     counts = dict(counts)
#     # add the extra features, for now just adding one count for each extra feature
#     for feature in extra_features:
#         counts[feature] += 1
#     return counts 



from collections import Counter
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

def to_feature_vector_dictionary(character_doc, extra_features=None, ngram_type="unigrams"):
    if extra_features is None:
        extra_features = []
    
    # Step 1: Simple Token Counts
    counts = Counter(character_doc)

    # Step 2: Prepare input for TF-IDF
    string_tokens = [' '.join(ngram) if isinstance(ngram, tuple) else ngram for ngram in character_doc]

    # Generate TF-IDF scores
    tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(), lowercase=False)
    tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(string_tokens)])
    tfidf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]))

    # Combine token counts and TF-IDF scores
    feature_vector = {str(token): counts[token] + tfidf_scores.get(str(token), 0) for token in counts}

    # Step 3: Include extra features
    for feature in extra_features:
        feature_vector[str(feature)] = feature_vector.get(str(feature), 0) + 1

    # Step 4: Add sentiment analysis features
    blob = TextBlob(' '.join(string_tokens))
    feature_vector['sentiment_polarity'] = blob.sentiment.polarity
    feature_vector['sentiment_subjectivity'] = blob.sentiment.subjectivity

    # Step 5: Add n-gram features
    if ngram_type == "unigrams":
        ngrams_list = [token for token in character_doc if isinstance(token, str)]
    elif ngram_type == "bigrams":
        ngrams_list = [' '.join(ngram) for ngram in ngrams(character_doc, 2)]
    elif ngram_type == "trigrams":
        ngrams_list = [' '.join(ngram) for ngram in ngrams(character_doc, 3)]
    else:
        raise ValueError("Invalid ngram_type. Choose from 'unigrams', 'bigrams', or 'trigrams'.")

    for ngram in ngrams_list:
        feature_vector[f'ngram_{ngram}'] = feature_vector.get(f'ngram_{ngram}', 0) + 1

    return feature_vector

In [ ]:
corpusVectorizer = DictVectorizer()   # corpusVectorizor which will just produce sparse vectors from feature dicts
# Any matrix transformers (e.g. tf-idf transformers) should be initialized here


# def create_document_matrix_from_corpus(corpus, fitting=False):
#     """Method which fits different vectorizers
#     on data and returns a matrix.
    
#     Currently just does simple conversion to matrix by vectorizing the dictionary. Improve this for Q2.
    
#     ::corpus:: a list of (class_label, document) pairs.
#     ::fitting:: a boolean indicating whether to fit/train the vectorizers (should be true on training data)
#     """
    
#     # uses the global variable of the corpus Vectorizer to improve things
#     if fitting:
#         corpusVectorizer.fit([to_feature_vector_dictionary(doc, []) for name, doc in corpus])
#     doc_feature_matrix = corpusVectorizer.transform([to_feature_vector_dictionary(doc, []) for name, doc in corpus])
    
#     return doc_feature_matrix



def create_document_matrix_from_corpus(corpus, fitting=False):
    """Method which fits different vectorizers on data and returns a matrix."""
    feature_dicts = [to_feature_vector_dictionary(doc, []) for name, doc in corpus]
    if fitting:
        corpusVectorizer.fit(feature_dicts)

    raw_feature_matrix = corpusVectorizer.transform(feature_dicts)

    matrix = raw_feature_matrix.toarray()
    row_sum = matrix.sum(axis=1, keepdims=True)
    tf = matrix/np.maximum(row_sum, 1)

    doc_count = len(corpus)
    term_count = np.count_nonzero(matrix, axis=0)
    idf = np.log((doc_count / (1 + term_count)) + 1)

    tfid_matrix = tf * idf

    class SparseMatrixWrapper:
        def __init__(self, data):
            self.data = data

        def toarray(self):
            return self.data
        
    return SparseMatrixWrapper(tfid_matrix)

training_feature_matrix = create_document_matrix_from_corpus(training_corpus, fitting=True)

In [ ]:
training_feature_matrix

In [ ]:
# get the validation data- only 50 lines used for each character
val_character_docs = create_character_docs(val_data, max_line_count=50)
print('Num. Characters: ',len(val_character_docs.keys()),"\n")
total_words = 0
for name in val_character_docs.keys():
    print(name, 'Num of Words: ',len(val_character_docs[name].split()))
    total_words += len(val_character_docs[name].split())
print("total words", total_words)

# create list of pairs of (character name, pre-processed character) 
val_corpus = [(name, pre_process(doc)) for name, doc in sorted(val_character_docs.items())]
val_labels = [name for name, doc in val_corpus]

In [25]:
# Just transform the val_feature_matrix, don't fit
val_feature_matrix = create_document_matrix_from_corpus(val_corpus, fitting=False)

In [ ]:
val_feature_matrix

In [27]:
def compute_cosine_similarity(v1, v2):
    """Takes a pair of vectors v1 and v2 (1-d arrays e.g. [0, 0.5, 0.5])
    returns the cosine similarity between the vectors
    """
    
    # compute cosine similarity manually
    manual_cosine_similarity = np.dot(v1, v2)  /(norm(v1) * norm(v2))
    
    return manual_cosine_similarity

In [28]:
def compute_IR_evaluation_scores(train_feature_matrix, test_feature_matrix, train_labels, test_labels):
    
    rankings = []
    all_cosine_similarities = []
    pairwise_cosine_similarity = []
    pairs = []
    correct = 0
    for i, target in enumerate(test_labels):
        # compare the left out character against the mean
        idx = i 
        fm_1 = test_feature_matrix.toarray()[idx]
        all_sims = {}
        # print("target:", target)
        for j, other in enumerate(train_labels):
            fm_2 = train_feature_matrix.toarray()[j]
            manual_cosine_similarity = compute_cosine_similarity(fm_1, fm_2)
            pairs.append((target, other))
            pairwise_cosine_similarity.append(manual_cosine_similarity)
            if other == target:
                all_cosine_similarities.append(manual_cosine_similarity)
            all_sims[other] = manual_cosine_similarity

            # print(target, other, manual_cosine_similarity)
        sorted_similarities = sorted(all_sims.items(),key=lambda x:x[1],reverse=True)
        # print(sorted_similarities)
        ranking = {key[0]: rank for rank, key in enumerate(sorted_similarities, 1)}
        # print("Ranking for target", ranking[target])
        if ranking[target] == 1:
            correct += 1
        rankings.append(ranking[target])
        # print("*****")
    mean_rank = np.mean(rankings)
    mean_cosine_similarity = np.mean(all_cosine_similarities)
    accuracy = correct/len(test_labels)
    print("mean rank", np.mean(rankings))
    print("mean cosine similarity", mean_cosine_similarity)
    print(correct, "correct out of", len(test_labels), "/ accuracy:", accuracy )
    
    # get a dafaframe showing all the similarity scores of training vs test docs
    df = pd.DataFrame({'doc1': [x[0] for x in pairs], 'doc2': [x[1] for x in pairs],
                       'similarity': pairwise_cosine_similarity})

    # display characters which are most similar and least similar
    df.loc[[df.similarity.values.argmax(), df.similarity.values.argmin()]]
    return (mean_rank, mean_cosine_similarity, accuracy, df)

In [29]:
def plot_heat_map_similarity(df):
    """Takes a dataframe with header 'doc1, doc2, similarity'
    Plots a heatmap based on the similarity scores.
    """
    test_labels =  sorted(list(set(df.sort_values(['doc1'])['doc1'])))
    # add padding 1.0 values to either side
    cm = [[1.0,] * (len(test_labels)+2)]
    for target in test_labels:
        new_row = [1.0]
        for x in df.sort_values(['doc1', 'doc2'])[df['doc1']==target]['similarity']:
            new_row.append(x)
        new_row.append(1.0)
        cm.append(new_row)
    cm.append([1.0,] * (len(test_labels)+2))
    #print(cm)
    labels = [""] + test_labels + [""]
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Similarity matrix between documents as vectors')
    fig.colorbar(cax)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels( labels, rotation=45)
    ax.set_yticklabels( labels)

    for i in range(len(cm)):
        for j in range(len(cm)):

            text = ax.text(j, i, round(cm[i][j],3),
                           ha="center", va="center", color="w")

    plt.xlabel('Training Vector Doc')
    plt.ylabel('Test Vector Doc')
    #fig.tight_layout()
    plt.show()

In [ ]:
mean_rank, mean_cosine_simliarity, acc, df = compute_IR_evaluation_scores(training_feature_matrix, val_feature_matrix, train_labels, val_labels)

In [ ]:
plot_heat_map_similarity(df)

# Q1. Improve pre-processing (20 marks)
Using the pre-processing techniques you have learned in the module, improve the `pre_process` function above, which currently just tokenizes text based on white space.

When developing, use the first 300 and 50 lines from the training and validation files, as per above. To check the improvements by using the different techniques, use the `compute_IR_evaluation_scores` function as above. The **mean rank** is the main metric you need to focus on improving throughout this assignment, where the target/best possible performance is **1** (i.e. all test/validation data character documents are closest to their corresponding training data character documents) and the worst is **16**. Initially, the code in this template achieves a mean rank of **4.3**  and accuracy of **0.25** on the test set and a mean rank of **3.6** and accuracy of **0.31** on the validation set - you should be looking to improve those, particularly getting the mean rank as close to 1 as possible.


In [32]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import string

def pre_process(character_text):
    """Pre-process all the concatenated lines of a character."""
    # 1. Lowercasing
    character_text = character_text.lower()

    # 2: Separate the punctuations from words
    character_text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", character_text)
    character_text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", character_text)

    # 3. Tokenization (simple whitespace split)
    tokens = character_text.split()

    # 4. Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Punctuation Removal
    tokens = [word for word in tokens if word not in string.punctuation]

    # # Ensure tokens are not empty
    # if not tokens:
    #     tokens = ['placeholder']

    # # 6. Adding Bigrams and Trigrams
    # bigrams = list(ngrams(tokens, 2))
    # trigrams = list(ngrams(tokens, 3))

    # # Flatten bigrams and trigrams into a list of strings
    # bigrams = [' '.join(bigram) for bigram in bigrams]
    # trigrams = [' '.join(trigram) for trigram in trigrams]

    # # Combine bigrams and trigrams with tokens
    # tokens += bigrams
    # tokens += trigrams

    return tokens

# Q2. Improve linguistic feature extraction (30 marks)
Use the feature extraction techniques you have learned to improve the `to_feature_vector_dictionary` and `create_document_matrix_from_corpus` functions above. Examples of extra features could include extracting n-grams of different lengths and including POS tags. You could also use sentiment analysis or another text classifier's result when applied to the features for each character document. You could even use a gender classifier trained on the same data using the GENDER column **(but DO NOT USE the GENDER column directly in the features for the final vector)**.

Matrix transformation techniques like TF-IDF (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) are commonly used to improve the `create_document_matrix_from_corpus` function, which currently only uses a dictionary vectorizer (`DictVectorizer`) which straight-forwardly maps from the feature dictionaries produced for each character document to a sparse matrix.

Other options include using feature selection/reduction with techniques like minimum/maximum document frequency and/or feature selection like k-best selection using different statistical tests https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html. 

Again, develop your system using the training and validation sets and note the effect/improvement in mean rank with the techniques you use.

In [33]:
from collections import Counter
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

def to_feature_vector_dictionary(character_doc, extra_features=None, ngram_type="unigrams"):
    if extra_features is None:
        extra_features = []
    
    # Step 1: Simple Token Counts
    counts = Counter(character_doc)

    # Step 2: Prepare input for TF-IDF
    string_tokens = [' '.join(ngram) if isinstance(ngram, tuple) else ngram for ngram in character_doc]

    # Generate TF-IDF scores
    tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(), lowercase=False)
    tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(string_tokens)])
    tfidf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0]))

    # Combine token counts and TF-IDF scores
    feature_vector = {str(token): counts[token] + tfidf_scores.get(str(token), 0) for token in counts}

    # Step 3: Include extra features
    for feature in extra_features:
        feature_vector[str(feature)] = feature_vector.get(str(feature), 0) + 1

    # Step 4: Add sentiment analysis features
    blob = TextBlob(' '.join(string_tokens))
    feature_vector['sentiment_polarity'] = blob.sentiment.polarity
    feature_vector['sentiment_subjectivity'] = blob.sentiment.subjectivity

    # Step 5: Add n-gram features
    if ngram_type == "unigrams":
        ngrams_list = [token for token in character_doc if isinstance(token, str)]
    elif ngram_type == "bigrams":
        ngrams_list = [' '.join(ngram) for ngram in ngrams(character_doc, 2)]
    elif ngram_type == "trigrams":
        ngrams_list = [' '.join(ngram) for ngram in ngrams(character_doc, 3)]
    else:
        raise ValueError("Invalid ngram_type. Choose from 'unigrams', 'bigrams', or 'trigrams'.")

    for ngram in ngrams_list:
        feature_vector[f'ngram_{ngram}'] = feature_vector.get(f'ngram_{ngram}', 0) + 1

    return feature_vector

In [ ]:
corpusVectorizer = DictVectorizer()   # corpusVectorizor which will just produce sparse vectors from feature dicts

def create_document_matrix_from_corpus(corpus, fitting=False):
    """Method which fits different vectorizers on data and returns a matrix."""
    feature_dicts = [to_feature_vector_dictionary(doc, []) for name, doc in corpus]
    if fitting:
        corpusVectorizer.fit(feature_dicts)

    raw_feature_matrix = corpusVectorizer.transform(feature_dicts)

    matrix = raw_feature_matrix.toarray()
    row_sum = matrix.sum(axis=1, keepdims=True)
    tf = matrix/np.maximum(row_sum, 1)

    doc_count = len(corpus)
    term_count = np.count_nonzero(matrix, axis=0)
    idf = np.log((doc_count / (1 + term_count)) + 1)

    tfid_matrix = tf * idf

    class SparseMatrixWrapper:
        def __init__(self, data):
            self.data = data

        def toarray(self):
            return self.data
        
    return SparseMatrixWrapper(tfid_matrix)

training_feature_matrix = create_document_matrix_from_corpus(training_corpus, fitting=True)

# Q3. Add dialogue context and scene features (15 marks)
Adjust `create_character_document_from_dataframe` and the other functions appropriately so the data incorporates the context of the line spoken by the characters in terms of the lines spoken by other characters in the same scene (before and after the target character's lines). HINT: you should use the *Episode* and *Scene* columns to check which characters are in the same scene to decide whether to include their lines or not. Only the lines from the same *Scene* can be added as the context as lines from different *Scene* are irrelevant.  **(but DO NOT USE the GENDER and CHARACTER columns directly)**.

In [35]:
def create_character_docs(df, max_line_count):
    
    character_docs = {}
    character_line_count = {}

    for (episode, scene), scene_data in df.groupby(['Episode', 'Scene']):
        scene_data = scene_data.sort_index()
        scene_characters = set(scene_data['Character_name'])

        for _, row in scene_data.iterrows():
            line = row['Line']
            name = row['Character_name']

            # Robust line cleaning
            if isinstance(line, float) and math.isnan(line):
                continue
            line = str(line).strip()  # Remove leading/trailing whitespace
            if not line:
                continue

            # Contextual filtering: Only include lines if the character interacts
            # with at least one other character in the scene
            if len(scene_characters) > 1:
                if name not in character_docs:
                    character_docs[name] = ""
                    character_line_count[name] = 0
                if character_line_count[name] < max_line_count:
                    character_docs[name] += line + " EOL "
                    character_line_count[name] += 1

    print("lines per character", character_line_count)
    return character_docs


def create_character_document_from_dataframe(df, max_line_count):
    
    character_docs = {}
    character_line_count = {}

    for (episode, scene), scene_data in df.groupby(['Episode', 'Scene']):
        scene_data = scene_data.sort_index()
        scene_characters = set(scene_data['Character_name'])

        for _, row in scene_data.iterrows():
            line = row['Line']
            name = row['Character_name']

            # Robust line cleaning
            if isinstance(line, float) and math.isnan(line):
                continue
            line = str(line).strip()  # Remove leading/trailing whitespace
            if not line:
                continue

            # Contextual filtering: Only include lines if the character interacts
            # with at least one other character in the scene
            if len(scene_characters) > 1:
                if name not in character_docs:
                    character_docs[name] = ""
                    character_line_count[name] = 0
                if character_line_count[name] < max_line_count:
                    character_docs[name] += line + " EOL "
                    character_line_count[name] += 1

    print("lines per character", character_line_count)
    return character_docs

# Q4. Parameter Search (15 marks)
It is a good practice to conduct a systematic parameter search instead of a random search as this will give you more 
reliable results. Given the scope of this assignment, it is possible to conduct a **grid search** on options you decided to try within the individual questions. The grid search should be done within the individual questions (i.e. Q1-Q3), and the later question should adopt the best settings from the previous questions. There is no need to do a grid search over all configurations from all questions as this will easily make the search unrealistic. E.g. Suppose we need 32, 90, and 4 runs to finish the grid search within questions, a cross-question grid search would need 32x90x4 = 11520 runs!                                                                      

In [36]:
from sklearn.model_selection import ParameterGrid
from sklearn.feature_extraction.text import TfidfVectorizer

def grid_search_tfidf(training_corpus, train_labels, test_corpus, test_labels):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import ParameterGrid
    
    param_grid = {
        'ngram_range': [(1, 1), (1, 2), (1, 3)],
        'max_df': [0.9, 0.8, 0.7],
        'min_df': [1, 2, 5],
        'use_idf': [True, False],
        'sublinear_tf': [True, False]
    }

    best_mean_rank = float('inf')
    best_params = None
    best_vectorizer = None

    for params in ParameterGrid(param_grid):
        print(f"Testing parameters: {params}")
        tfidf_vectorizer = TfidfVectorizer(**params)
        tfidf_vectorizer.fit([' '.join(doc) for name, doc in training_corpus])
        train_matrix = tfidf_vectorizer.transform([' '.join(doc) for name, doc in training_corpus])
        test_matrix = tfidf_vectorizer.transform([' '.join(doc) for name, doc in test_corpus])
        
        mean_rank, _, _, _ = compute_IR_evaluation_scores(train_matrix, test_matrix, train_labels, test_labels)  # Replace with your own scoring function
        if mean_rank < best_mean_rank:
            best_mean_rank = mean_rank
            best_params = params
            best_vectorizer = tfidf_vectorizer

    print("\nBest Parameters Found:")
    print(f"Mean Rank: {best_mean_rank}")
    print(f"Parameters: {best_params}")
    
    return best_vectorizer


In [ ]:
train_character_docs = create_character_document_from_dataframe(train_data, max_line_count=300)
training_corpus = [(name, pre_process(doc)) for name, doc in sorted(train_character_docs.items())]
train_labels = [name for name, doc in training_corpus]

test_character_docs = create_character_document_from_dataframe(test_data, max_line_count=50)
test_corpus = [(name, pre_process(doc)) for name, doc in sorted(test_character_docs.items())]
test_labels = [name for name, doc in test_corpus]

best_vectorizer = grid_search_tfidf(training_corpus, train_labels, test_corpus, test_labels)

train_feature_matrix = best_vectorizer.transform([' '.join(doc) for name, doc in training_corpus])
test_feature_matrix = best_vectorizer.transform([' '.join(doc) for name, doc in test_corpus])

mean_rank, mean_cosine_similarity, acc, df = compute_IR_evaluation_scores(
    train_feature_matrix, test_feature_matrix, train_labels, test_labels
)

print("\nFinal Test Results:")
print(f"Mean Rank: {mean_rank}")
print(f"Mean Cosine Similarity: {mean_cosine_similarity}")
print(f"Accuracy: {acc}")


# Q5. Analyse the similarity results (10 marks)
From your system so far run on the training/validation sets, identify the heldout character vectors ranked closest to each character's training vector which are not the character themselves, and those furthest away, as displayed using the `plot_heat_map_similarity` function. In your report, try to ascribe reasons why this is the case, particularly for those where there isn't a successful highest match between the target character in the training set and that character's vector in the heldout set yet. Observations you could make include how their language use is similar, resulting in similar word or ngram features.

#### Step 1: Define a Function to Find Closest and Furthest Vectors

In [38]:
def find_closest_and_furthest_vectors(train_feature_matrix, test_feature_matrix, train_labels, test_labels):
    """
    Finds the closest and furthest vectors for each training character.
    
    Parameters:
    - train_feature_matrix: Feature matrix of training data.
    - test_feature_matrix: Feature matrix of test data.
    - train_labels: List of character names for the training data.
    - test_labels: List of character names for the test data.
    
    Returns:
    - closest_vectors: List of tuples (train_character, closest_test_character).
    - furthest_vectors: List of tuples (train_character, furthest_test_character).
    """
    closest_vectors = []
    furthest_vectors = []

    # Iterate through each character in the training set
    for train_char in train_labels:
        # Filter the dataframe for the current character
        char_df = df[df['doc1'] == train_char]
        
        # Rank based on similarity
        ranked_df = char_df.sort_values(by='similarity', ascending=False)
        
        # Skip the first row (self-similarity) and find closest/furthest vectors
        closest_vector = ranked_df.iloc[1]['doc2']  # Closest vector (not itself)
        furthest_vector = ranked_df.iloc[-1]['doc2']  # Furthest vector
        
        closest_vectors.append((train_char, closest_vector))
        furthest_vectors.append((train_char, furthest_vector))
    
    return closest_vectors, furthest_vectors


#### Step 2: Compute Similarity Scores and Analyze Results

In [ ]:
# Step 1: Compute IR evaluation scores
mean_rank, mean_cosine_similarity, acc, df = compute_IR_evaluation_scores(
    training_feature_matrix, val_feature_matrix, train_labels, val_labels
)

# Step 2: Find closest and furthest vectors
closest_vectors, furthest_vectors = find_closest_and_furthest_vectors(
    training_feature_matrix, val_feature_matrix, train_labels, val_labels
)

# Step 3: Print closest and furthest vectors
print("Closest Vectors:")
print(closest_vectors)

print("\nFurthest Vectors:")
print(furthest_vectors)


#### Step 3: Heatmap Visualization

In [40]:
def plot_heat_map_similarity(df):
    """
    Plots a heatmap based on the similarity scores.
    
    Parameters:
    - df: DataFrame with columns 'doc1', 'doc2', and 'similarity'.
    """
    test_labels = sorted(list(set(df.sort_values(['doc1'])['doc1'])))
    
    # Add padding (1.0) for heatmap
    cm = [[1.0] * (len(test_labels) + 2)]
    for target in test_labels:
        new_row = [1.0]
        for x in df.sort_values(['doc1', 'doc2'])[df['doc1'] == target]['similarity']:
            new_row.append(x)
        new_row.append(1.0)
        cm.append(new_row)
    cm.append([1.0] * (len(test_labels) + 2))
    
    labels = [""] + test_labels + [""]
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Similarity matrix between documents as vectors')
    fig.colorbar(cax)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=45)
    ax.set_yticklabels(labels)
    
    for i in range(len(cm)):
        for j in range(len(cm)):
            text = ax.text(j, i, round(cm[i][j], 3),
                           ha="center", va="center", color="w")

    plt.xlabel('Training Vector Doc')
    plt.ylabel('Test Vector Doc')
    plt.show()


In [ ]:
plot_heat_map_similarity(df)

# Q6. Run on final test data  (10 marks)
Test your best system using the code below to train on the training data (using the first 300 lines per character maximum) and do the final testing on the test file (using the first 50 lines per character maximum).

Make any necessary adjustments such that it runs in the same way as the training/testing regime you developed above- e.g. making sure any transformer objects are initialized before `create_document_matrix_from_corpus` is called. Make sure your best system is left in the notebook and it is clear what the mean rank and accuracy of document selection are on the test data.

In [ ]:
# redo on training data with the first 300 character lines used
train_character_docs = create_character_document_from_dataframe(train_data, max_line_count=300)
print('Num. Characters: ',len(train_character_docs.keys()),"\n")
total_words = 0
for name in train_character_docs.keys():
    print(name, 'Number of Words: ',len(train_character_docs[name].split()))
    total_words += len(train_character_docs[name].split())
print("total words", total_words)

training_corpus = [(name, pre_process(doc)) for name, doc in sorted(train_character_docs.items())]
train_labels = [name for name, doc in training_corpus]

training_feature_matrix = create_document_matrix_from_corpus(training_corpus, fitting=True)

# get the test data using 50 lines per character
test_character_docs = create_character_document_from_dataframe(test_data, max_line_count=50)
print('Num. Characters: ',len(test_character_docs.keys()),"\n")
total_words = 0
for name in test_character_docs.keys():
    print(name, 'Number of Words: ',len(test_character_docs[name].split()))
    total_words += len(test_character_docs[name].split())
print("total words", total_words)

# create list of pairs of (character name, pre-processed character) 
test_corpus = [(name, pre_process(doc)) for name, doc in sorted(test_character_docs.items())]
test_labels = [name for name, doc in test_corpus]


# Just transform the val_feature_matrix, don't fit
test_feature_matrix = create_document_matrix_from_corpus(test_corpus, fitting=False)


mean_rank, mean_cosine_simliarity, acc, df = compute_IR_evaluation_scores(training_feature_matrix, test_feature_matrix, train_labels, test_labels)